In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    06/11/2025  -  23:55:40           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Competición Perretes y Gatos

## Iteración 5 - Fine-tuning VGG16

**Situación actual**: Iteración 5 con Transfer Learning consiguió 0.86380 (posición #7).

**Pasos que voy a hacer con Fine-tuning**:
- VGG16 pre-entrenado en ImageNet (14M imágenes)
- Descongelar las últimas 4 capas convolucionales (block5)
- Learning rate MUY bajo (1e-5) para no romper pesos pre-entrenados
- Solo 10 épocas adicionales con ajuste fino
- Objetivo: +2-4% score → 0.88-0.90

**kaggle score**: 0.89552, lo cual mejora respecto al anterior -> siguiente planteamiento aplicar esemble con 3 modelos

In [ ]:
# Importo las librerías necesarias para trabajar con el modelo y los datos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

# Establezco una semilla aleatoria para que los resultados sean reproducibles
seed = 42
keras.utils.set_random_seed(seed)

# Rutas de datos - cambia solo el nombre del dataset si es diferente
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Versión de Keras:", keras.__version__)
print("Rutas configuradas correctamente")

## Carga y Preparación de los Datos

Cargo las imágenes del conjunto de entrenamiento y las divido automáticamente en dos partes:
- **80% para entrenamiento**: El modelo aprende de estas imágenes
- **20% para validación**: Evalúo el rendimiento en cada época sin que el modelo las haya visto

Esta división me permite detectar si el modelo está memorizando (overfitting) en lugar de
aprender patrones generalizables. Todas las imágenes se redimensionan a **224x224 píxeles**,
que es el tamaño de entrada estándar requerido por VGG16.

El modo `binary` es crucial para clasificación de 2 clases, ya que genera etiquetas 0/1
en lugar de vectores one-hot, lo que simplifica la salida y evita problemas de calibración.

In [ ]:
# Tamaño de imagen - 224x224 para VGG16
image_size = (224, 224)

# Batch size
batch_size = 125

# Cargo las imágenes del directorio de entrenamiento
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Cambio a binary para más estabilidad
)

print(f"Tamaño del conjunto de entrenamiento: {len(train_ds)} batches")
print(f"Tamaño del conjunto de validación: {len(val_ds)} batches")
print(f"Total imágenes de entrenamiento aproximadas: {len(train_ds) * batch_size}")
print(f"Total imágenes de validación aproximadas: {len(val_ds) * batch_size}")

## Data Augmentation (Aumento de Datos)

Aplico transformaciones aleatorias a las imágenes **solo durante el entrenamiento** para
hacer el modelo más robusto y evitar que memorice. Estas transformaciones simulan variaciones
naturales que puede encontrar el modelo en datos reales:

- **RandomFlip horizontal**: Voltea la imagen (perros/gatos pueden estar orientados de cualquier forma)
- **RandomRotation (±36°)**: Rotaciones suaves que no distorsionan la imagen
- **RandomZoom (±10%)**: Acerca o aleja la imagen ligeramente
- **RandomTranslation (±10%)**: Desplaza la imagen horizontal y verticalmente

He configurado transformaciones **conservadoras** (valores bajos) porque VGG16 ya conoce
características generales de millones de imágenes. No necesito augmentation agresivo como
en un modelo entrenado desde cero, solo variaciones sutiles para mejorar la generalización.

In [ ]:
# Secuencia de transformaciones aleatorias aplicadas solo durante entrenamiento
# Más conservador para Transfer Learning
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),  # ±36° suficiente
    keras.layers.RandomZoom(0.1),      # ±10% sin distorsión
    keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
], name="data_augmentation")

print("Data Augmentation configurado (modo conservador para Transfer Learning)")

## Construcción del Modelo - Transfer Learning

**Transfer Learning** es la técnica de reutilizar un modelo pre-entrenado en un problema similar
y adaptarlo a nuestro caso específico. En lugar de entrenar desde cero (muy lento y requiere
millones de imágenes), aprovecho VGG16, que ya aprendió características visuales generales
entrenándose con ImageNet (14 millones de imágenes, 1000 clases).

**Arquitectura del modelo:**
1. **Base congelada (VGG16)**: Todas las capas convolucionales permanecen fijas con sus pesos
   pre-entrenados. Estas capas ya saben detectar bordes, texturas, formas y patrones complejos.
   
2. **Cabecera personalizada** (lo único que entreno):
   - Global Average Pooling: Resume los feature maps en un vector compacto
   - Dense(256, relu): Capa de 256 neuronas para combinar características
   - Dropout(0.5): Desactiva aleatoriamente el 50% de neuronas para evitar overfitting
   - Dense(1, sigmoid): Salida binaria que produce una probabilidad entre 0 (Cat) y 1 (Dog)

Esta estrategia es mucho más eficiente: entreno solo ~67K parámetros en lugar de ~15M,
y consigo excelente precisión en minutos en vez de días.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.applications import VGG16

# Forma de entrada: 224x224 con 3 canales RGB (requerido por VGG16)
input_shape = image_size + (3,)

# Cargo VGG16 pre-entrenado sin la parte superior (sin clasificación)
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape,
    pooling='avg'  # Global Average Pooling
)

# CONGELO todas las capas de VGG16 - no las voy a entrenar
base_model.trainable = False

# Construyo el modelo completo
model = Sequential([
    keras.Input(shape=input_shape),
    data_augmentation,  # Augmentation solo en entrenamiento
    base_model,         # VGG16 congelado
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Salida binaria
], name='VGG16_Transfer_Learning')

print(f"Modelo Transfer Learning creado")
print(f"Capas VGG16 congeladas: {len(base_model.layers)}")
print(f"Solo entrenaremos: Dense(256) + Dropout + Dense(1)")
model.summary()

## Compilación y Entrenamiento - Transfer Learning

Configuro el proceso de entrenamiento con hiperparámetros adaptados a Transfer Learning:

**Optimizer: Adam con learning rate = 0.0001**
- Adam es adaptativo y converge más rápido que SGD
- Learning rate bajo (0.0001) porque los pesos pre-entrenados ya son buenos y no quiero
  hacer cambios bruscos. Solo ajusto suavemente la cabecera personalizada.

**Loss: Binary Crossentropy**
- Función de pérdida estándar para clasificación binaria
- Penaliza las predicciones incorrectas de forma logarítmica

**Métricas: Accuracy, Precision, Recall**
- Accuracy: % de predicciones correctas (métrica principal)
- Precision: De las predicciones positivas, cuántas son correctas (evitar falsos positivos)
- Recall: De todos los positivos reales, cuántos detectamos (evitar falsos negativos)

**ReduceLROnPlateau callback:**
- Si la validación se estanca durante 3 épocas, reduce el learning rate a la mitad
- Esto ayuda a refinar los pesos cuando el modelo se acerca al óptimo
- Learning rate mínimo: 1e-7

**Épocas: 15**
Transfer Learning converge rápido porque la base ya está entrenada. Con 15 épocas
es suficiente para ajustar la cabecera personalizada sin sobreajustar.

In [ ]:
%%time

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # LR bajo para Transfer Learning
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

epochs = 15  # Transfer Learning converge rápido

# ReduceLROnPlateau: reduce el learning rate si se estanca
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

print("Configuración Transfer Learning VGG16:")
print(f"Épocas: {epochs}")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Modelo base: VGG16 ImageNet (congelado)")
print(f"Entrenamos: Solo 2 capas Dense")
print("-" * 60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[reduce_lr],
    verbose=1
)

print("-" * 60)
print(f"Entrenamiento completado en {len(history.history['loss'])} épocas")
print(f"Val Accuracy final: {history.history['val_accuracy'][-1]:.4f}")
print(f"Val Precision final: {history.history['val_precision'][-1]:.4f}")
print(f"Val Recall final: {history.history['val_recall'][-1]:.4f}")

## Visualización de las Curvas de Aprendizaje

Estos gráficos son fundamentales para diagnosticar el comportamiento del modelo durante
el entrenamiento y detectar problemas antes de hacer predicciones:

**Gráfico de Pérdida (Loss):**
- **Esperado**: Ambas curvas (train y val) descienden y convergen
- **Overfitting**: Pérdida de entrenamiento muy baja, pérdida de validación alta o creciente
- **Underfitting**: Ambas curvas se estancan en valores altos

**Gráfico de Precisión (Accuracy):**
- **Esperado**: Ambas curvas crecen y se estabilizan en valores altos (>90%)
- **Overfitting**: Precisión de entrenamiento muy alta (>99%), validación estancada o bajando
- **Underfitting**: Ambas precisiones bajas (<70%)

**Interpretación saludable para Transfer Learning:**
- Las curvas deben estar muy cercanas (diferencia <2-3%)
- La validación puede incluso ser ligeramente superior al entrenamiento (indica buena generalización)
- Si la validación es muy inferior, necesito más regularización (dropout, augmentation)

In [ ]:
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))

# Gráfico de pérdida
plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='Pérdida en entrenamiento')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='Pérdida en validación')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Evolución de la Pérdida")
plt.legend()
plt.grid(True, alpha=0.3)

# Gráfico de precisión
plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='Precisión en entrenamiento')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='Precisión en validación')
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.title("Evolución de la Precisión")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("RESULTADOS FINALES:")
print("="*60)
print(f"Precisión final en entrenamiento: {logs['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs['val_accuracy'].iloc[-1]:.4f}")
print(f"Pérdida final en entrenamiento: {logs['loss'].iloc[-1]:.4f}")
print(f"Pérdida final en validación: {logs['val_loss'].iloc[-1]:.4f}")
print("="*60)

## Guardado del Modelo

Guardo el modelo entrenado para poder reutilizarlo más tarde sin tener que volver a entrenarlo.
Keras guarda toda la arquitectura, los pesos aprendidos y la configuración de compilación.

In [ ]:
model.save("model.keras")
print("Modelo guardado como 'model.keras'")

## Evaluación con Datos Suplementarios

Los **datos suplementarios** son un conjunto adicional proporcionado por la competición
para estimar el rendimiento del modelo en datos completamente nuevos, simulando el
conjunto de test real.

**¿Por qué es importante?**
- El conjunto de validación (20% del training) puede no ser totalmente representativo
- Los datos suplementarios me dan una estimación más realista del score en Kaggle
- Si el modelo funciona bien aquí, probablemente funcionará bien en el leaderboard

**Comparación con iteraciones anteriores:**
Esta métrica es la que uso para decidir si una modificación (como el fine-tuning)
realmente mejora el modelo o solo memoriza mejor el conjunto de validación.

In [ ]:
supplementary_ds = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Binary consistente
)

print("Evaluando con datos suplementarios...")
print("-" * 60)

results = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print("-" * 60)
print("\nRESULTADOS EN DATOS SUPLEMENTARIOS:")
print(f"Precisión: {results['accuracy']:.4f}")
print(f"Pérdida: {results['loss']:.4f}")
print("-" * 60)

---

## ITERACIÓN 6 - Fine-tuning VGG16

**Situación actual**: Iteración 5 con Transfer Learning consiguió 0.86380 (posición #7).

**Estrategia de mejora**:
- Descongelar las últimas 4 capas convolucionales de VGG16 (block5)
- Learning rate MUY bajo (1e-5) para no romper los pesos pre-entrenados
- Entrenar 10 épocas adicionales con ajuste fino
- Objetivo: +2-4% de score → 0.88-0.90

**Riesgo**: Puede empeorar si sobreajustamos. Por eso usamos LR muy bajo y pocas épocas.

**Referencias**:
- CS231n Fine-tuning: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf (slides 35-40)
- Keras Fine-tuning guide: https://keras.io/guides/transfer_learning/#fine-tuning


In [ ]:
# Verifico la estructura de VGG16 para saber qué descongelar
print("Estructura de capas de VGG16:")
print("=" * 70)

for i, layer in enumerate(base_model.layers):
    print(f"Capa {i:2d}: {layer.name:20s} - Entrenable: {layer.trainable}")

print("\n" + "=" * 70)
print("VGG16 tiene 5 bloques convolucionales (block1 a block5)")
print("Vamos a descongelar SOLO el block5 (últimas 4 capas: 15-18)")
print("=" * 70)

In [ ]:
# DESCONGELO solo el último bloque (block5) de VGG16
# Son las 4 últimas capas convolucionales (capas 15-18)

base_model.trainable = True  # Primero habilito el entrenamiento

# Ahora congelo TODO excepto block5
for layer in base_model.layers[:-4]:  # Todas menos las últimas 4
    layer.trainable = False

# Verifico qué quedó descongelado
print("Estado después de descongelar block5:")
print("=" * 70)
trainable_count = 0
frozen_count = 0

for i, layer in enumerate(base_model.layers):
    status = "✓ ENTRENABLE" if layer.trainable else "  (congelada)"
    print(f"Capa {i:2d}: {layer.name:20s} {status}")
    if layer.trainable:
        trainable_count += 1
    else:
        frozen_count += 1

print("=" * 70)
print(f"Capas congeladas: {frozen_count}")
print(f"Capas entrenables: {trainable_count}")
print(f"Total parámetros: {model.count_params():,}")
print("=" * 70)

## Recompilación con Learning Rate MUY bajo para Fine-tuning

Ahora que he descongelado las últimas 4 capas convolucionales de VGG16, necesito
recompilar el modelo con un **learning rate mucho más bajo** que en el Transfer Learning.

**¿Por qué un learning rate tan bajo?**
Los pesos de VGG16 fueron entrenados con 14 millones de imágenes de ImageNet y son
extremadamente valiosos. Si uso un learning rate normal (1e-4), podría destruir estos
pesos pre-entrenados que ya funcionan muy bien.

**Comparación de learning rates:**
- **Transfer Learning (cabecera)**: 1e-4 (0.0001) → Ajuste rápido de capas random
- **Fine-tuning (block5)**: 1e-5 (0.00001) → Ajuste suave de capas pre-entrenadas (10x más bajo)

Este learning rate extremadamente bajo hace que las actualizaciones sean microscópicas,
permitiendo que el modelo se especialice en perros vs gatos sin olvidar los patrones
generales aprendidos de ImageNet.

**Callbacks ajustados:**
- **ReduceLROnPlateau**: Si se estanca 2 épocas, reduce LR a la mitad (más agresivo que antes)
- **EarlyStopping**: Si no mejora en 4 épocas, para el entrenamiento y restaura los mejores pesos
  (evita sobreajustar las capas convolucionales)

**Épocas: 10 máximo**
Pocas épocas son suficientes para ajuste fino. Más épocas podrían hacer que el modelo
olvide los features generales de ImageNet y se sobreajuste solo a nuestros gatos/perros.

In [ ]:
%%time

# Recompilo con learning rate MUY bajo para fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # 10x más bajo que antes
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print("Modelo recompilado para fine-tuning")
print(f"Learning rate: 1e-5 (0.00001)")
print(f"Optimizador: Adam")
print("=" * 70)

# Pocas épocas para no sobreajustar
epochs_finetuning = 10

# Callback para reducir LR si se estanca
reduce_lr_ft = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)

# Early stopping conservador para evitar sobreajuste
early_stop_ft = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True,
    verbose=1
)

print("Comenzando fine-tuning...")
print(f"Épocas máximas: {epochs_finetuning}")
print(f"Early stopping: patience=4 épocas")
print("=" * 70)

history_finetuning = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_finetuning,
    callbacks=[reduce_lr_ft, early_stop_ft],
    verbose=1
)

print("=" * 70)
print(f"Fine-tuning completado en {len(history_finetuning.history['loss'])} épocas")
print(f"Val Accuracy final: {history_finetuning.history['val_accuracy'][-1]:.4f}")
print(f"Val Precision final: {history_finetuning.history['val_precision'][-1]:.4f}")
print(f"Val Recall final: {history_finetuning.history['val_recall'][-1]:.4f}")
print(f"Val Loss final: {history_finetuning.history['val_loss'][-1]:.4f}")
print("=" * 70)

## Visualización del Fine-tuning

Estos gráficos muestran cómo evolucionó el modelo durante las épocas adicionales de
fine-tuning, después del entrenamiento inicial de Transfer Learning.

**Lo que busco en estos gráficos:**

1. **Mejora gradual**: Las métricas de validación deben mejorar o mantenerse estables,
   no empeorar. Si empeoran, significa que el fine-tuning está destruyendo los pesos
   pre-entrenados (sobreajuste catastrófico).

2. **Convergencia suave**: Las curvas deben ser relativamente estables, sin oscilaciones
   bruscas. El learning rate muy bajo (1e-5) garantiza esta suavidad.

3. **No degradación**: La pérdida de validación no debe aumentar significativamente.
   Un aumento indicaría que estoy sobreajustando las capas convolucionales.

**Señales de éxito del fine-tuning:**
- Val accuracy mejora respecto al Transfer Learning base
- Val loss se mantiene bajo o disminuye
- Train y val se mantienen cercanos (buena generalización)

**Señales de fracaso (sobreajuste):**
- Val accuracy baja o se estanca
- Val loss aumenta mientras train loss baja
- Gran separación entre curvas de train y val

In [ ]:
logs_ft = pd.DataFrame(history_finetuning.history)

plt.figure(figsize=(14, 4))

# Gráfico de pérdida
plt.subplot(1, 2, 1)
plt.plot(logs_ft["loss"], lw=2, label='Pérdida en entrenamiento', marker='o')
plt.plot(logs_ft["val_loss"], lw=2, label='Pérdida en validación', marker='o')
plt.xlabel("Época (Fine-tuning)")
plt.ylabel("Pérdida")
plt.title("Evolución de la Pérdida - Fine-tuning")
plt.legend()
plt.grid(True, alpha=0.3)

# Gráfico de precisión
plt.subplot(1, 2, 2)
plt.plot(logs_ft["accuracy"], lw=2, label='Precisión en entrenamiento', marker='o')
plt.plot(logs_ft["val_accuracy"], lw=2, label='Precisión en validación', marker='o')
plt.xlabel("Época (Fine-tuning)")
plt.ylabel("Precisión")
plt.title("Evolución de la Precisión - Fine-tuning")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("RESULTADOS FINE-TUNING:")
print("="*70)
print(f"Precisión final en entrenamiento: {logs_ft['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs_ft['val_accuracy'].iloc[-1]:.4f}")
print(f"Pérdida final en entrenamiento: {logs_ft['loss'].iloc[-1]:.4f}")
print(f"Pérdida final en validación: {logs_ft['val_loss'].iloc[-1]:.4f}")
print("="*70)

## Evaluación Final con Datos Suplementarios

Esta es la **métrica decisiva** para saber si el fine-tuning realmente mejoró el modelo
o si solo memorizó mejor el conjunto de validación.

**Comparación clave:**
- **Antes (Transfer Learning)**: Accuracy en supplementary = 72.33%
- **Después (Fine-tuning)**: Accuracy en supplementary = ?

**Interpretación de resultados:**

**Si mejora (+2% o más)**: El fine-tuning fue exitoso. Las capas convolucionales
  se especializaron en características de perros/gatos sin perder generalización.
  → ENVIAR submission.csv a Kaggle

**Si se mantiene (±1%)**: El fine-tuning no aportó mejora significativa, pero tampoco
  empeoró. El modelo base era suficientemente bueno.
  → Opcional enviarlo, probablemente score similar

**Si empeora (-2% o más)**: El fine-tuning sobreajustó el modelo. Destruyó features
  generales aprendidos de ImageNet.
  → NO enviar, mantener el modelo de Transfer Learning base

Esta evaluación en supplementary data simula cómo se comportará el modelo en el
leaderboard público de Kaggle, ya que son datos completamente nuevos que el modelo
nunca vio durante el entrenamiento.

In [ ]:
print("Evaluando modelo con fine-tuning en datos suplementarios...")
print("=" * 70)

results_ft = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print("=" * 70)
print("\nRESULTADOS DESPUES DE FINE-TUNING:")
print(f"Precisión: {results_ft['accuracy']:.4f}")
print(f"Precision: {results_ft['precision']:.4f}")
print(f"Recall: {results_ft['recall']:.4f}")
print(f"Pérdida: {results_ft['loss']:.4f}")
print("=" * 70)

# Comparación con Iteración 5 (antes del fine-tuning)
print("\nCOMPARACION CON ITERACION 5 (Transfer Learning sin fine-tuning):")
print("=" * 70)
print(f"Iter 5 - Supplementary Acc: {results['accuracy']:.4f}")
print(f"Iter 6 - Supplementary Acc: {results_ft['accuracy']:.4f}")
diff = results_ft['accuracy'] - results['accuracy']
print(f"Diferencia: {diff:+.4f} ({diff*100:+.2f}%)")
print("=" * 70)

if diff > 0:
    print("EXITO: Fine-tuning mejoró la generalización")
else:
    print("ALERTA: Fine-tuning no mejoró (posible sobreajuste)")

## Guardado del Modelo Fine-tuned

Guardo el modelo completo con fine-tuning para tener dos versiones disponibles:

1. **model.keras**: Modelo base de Transfer Learning (todas las capas VGG16 congeladas)
2. **model_finetuned.keras**: Modelo mejorado con fine-tuning (block5 especializado)

Esto me permite comparar ambos modelos y elegir el mejor para la submission final.
Keras guarda toda la arquitectura, los pesos entrenados y la configuración de compilación,
por lo que puedo cargar este modelo más tarde sin tener que volver a entrenar.

In [ ]:
model.save("model_finetuned.keras")
print("Modelo con fine-tuning guardado como 'model_finetuned.keras'")
print("(El modelo anterior sin fine-tuning sigue en 'model.keras')")

## Generación de Predicciones para Kaggle

Ahora genero las predicciones finales con el modelo optimizado mediante fine-tuning.

**¿Por qué usar el modelo fine-tuned?**
El fine-tuning permitió que las últimas capas convolucionales de VGG16 se especializaran
en distinguir características específicas de perros y gatos, mejorando la precisión en
datos nuevos (supplementary accuracy subió de 72.33% a 74.67%, un +2.33%).

Este proceso predice cada imagen del conjunto de test y genera el archivo `submission.csv`
que se enviará a Kaggle con el formato requerido (id, label).

In [ ]:
%%time

predictions_dict = {}

print(f"Generando predicciones para {len(os.listdir(TEST_PATH))} imágenes del conjunto de test...")
print("Usando umbral 0.5 para clasificación binaria (probabilidad >= 0.5 → Dog, < 0.5 → Cat)")
print("-" * 70)

# Itero sobre todas las imágenes del directorio de test
for img in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img)
    file_name = img_path.split('/')[-1]
    file_no_extension = file_name.split('.')[0]  # ID numérico de la imagen
    
    # Cargo la imagen y la preparo para el modelo
    img_loaded = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img_loaded)
    img_array = keras.ops.expand_dims(img_array, 0)  # Añado dimensión de batch
    
    # Predigo probabilidad con el modelo fine-tuned
    # Salida sigmoid: valor entre 0 (Cat) y 1 (Dog)
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    # Conversión explícita a clase binaria
    label = 1 if prediction >= 0.5 else 0
    
    predictions_dict[int(file_no_extension)] = label

print("-" * 70)
print(f"Predicciones completadas: {len(predictions_dict)} imágenes procesadas")

## Creación del Archivo de Submission para Kaggle

Genero el archivo CSV final con el formato exacto que requiere Kaggle:
- Columna `id`: número identificador de cada imagen
- Columna `label`: predicción (0 = Cat, 1 = Dog)

Además, verifico que la distribución de clases sea razonable (aproximadamente 50/50)
para asegurarme de que el modelo no está sesgado hacia una sola clase.

In [ ]:
# Creo el DataFrame con las predicciones y lo ordeno por ID
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"])
submission = submission.sort_values(by='id', ascending=True)

# Guardo el archivo CSV para Kaggle
submission.to_csv('submission.csv', index=False)

print("="*70)
print("ARCHIVO DE SUBMISSION CREADO: submission.csv")
print("="*70)

# Estadísticas de las predicciones
print("\nDISTRIBUCION DE PREDICCIONES:")
print("-" * 70)
print(submission["label"].value_counts())
print()
print(f"Clase 0 (Cat): {(submission['label'] == 0).sum():4d} imágenes ({100*(submission['label'] == 0).sum()/len(submission):5.1f}%)")
print(f"Clase 1 (Dog): {(submission['label'] == 1).sum():4d} imágenes ({100*(submission['label'] == 1).sum()/len(submission):5.1f}%)")
print(f"Total:         {len(submission):4d} imágenes")

# Muestro ejemplos de predicciones
print("\nPRIMERAS 10 PREDICCIONES:")
print(submission.head(10).to_string(index=False))

print("\nULTIMAS 10 PREDICCIONES:")
print(submission.tail(10).to_string(index=False))

# VERIFICACIÓN CRÍTICA: detectar si todas las predicciones son de una sola clase
print("\n" + "="*70)
if (submission['label'] == 0).sum() == len(submission) or (submission['label'] == 1).sum() == len(submission):
    print("ALERTA: TODAS LAS PREDICCIONES SON DE UNA SOLA CLASE")
    print("El modelo está completamente sesgado. NO enviar a Kaggle.")
    print("="*70)
else:
    print("VERIFICACION EXITOSA")
    print("  - Distribucion de clases balanceada")
    print("  - Archivo listo para enviar a Kaggle")
    print("  - Mejora esperada respecto a Iter 5: +2-4% (0.88-0.90)")
    print("="*70)